In [30]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
import xgboost as xgb

pd.set_option('display.max_columns',1000)

In [2]:
train = pd.read_csv("new_features/full_train_v3.csv")
test = pd.read_csv("new_features/full_test_v3.csv")

In [4]:
y_train = train['y'].values
y_mean = np.mean(y_train)
id_test = test['ID']

In [32]:
class StackingCVRegressorAveraged(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, regressors, meta_regressor, n_folds=5):
        self.regressors = regressors
        self.meta_regressor = meta_regressor
        self.n_folds = n_folds
        
    def fit(self, X, y):
        self.regr_ = [list() for x in self.regressors]
        self.meta_regr_ = clone(self.meta_regressor)
        
        from sklearn.model_selection import KFold
        kfolds = KFold(n_splits=self.n_folds, shuffle=True)
        
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.regressors)))
        
        for i, clf in enumerate(self.regressors):
            for train_idx, holdout_idx in kfold.split(X,y):
                instance = clone(clf)
                self.regr_[i].append(instance)
                
                instance.fit(X[train_idx], y[train_idx])
                y_pred = instance.predict(X[holdout_idx])
                out_of_fold_predictions[holdout_idx, i] = y_pred
                
        self.meta_regr_.fit(out_of_fold_predictions,y)
        
        return self
    
    def predict(self, X):
        meta_features = np.column_stack([
            
            np.column_stack([r.predict(X) for r in regrs]).mean(axis=1) for regrs in self.regr_
            
        ])
        return self.meta_regr_.predict(meta_features)

In [27]:
class StackingCVRegressorRetrained(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, regressors, meta_regressor, n_folds=5, use_features_in_secondary=False):
        self.regressors = regressors
        self.meta_regressor = meta_regressor
        self.n_folds = n_folds
        self.use_features_in_secondary = use_features_in_secondary

    def fit(self, X, y):
        self.regr_ = [clone(x) for x in self.regressors]
        self.meta_regr_ = clone(self.meta_regressor)

        kfold = KFold(n_splits=self.n_folds, shuffle=True)

        out_of_fold_predictions = np.zeros((X.shape[0], len(self.regressors)))

        # Create out-of-fold predictions for training meta-model
        for i, regr in enumerate(self.regr_):
            for train_idx, holdout_idx in kfold.split(X, y):
                instance = clone(regr)
                instance.fit(X[train_idx], y[train_idx])
                out_of_fold_predictions[holdout_idx, i] = instance.predict(X[holdout_idx])

        # Train meta-model
        if self.use_features_in_secondary:
            self.meta_regr_.fit(np.hstack((X, out_of_fold_predictions)), y)
        else:
            self.meta_regr_.fit(out_of_fold_predictions, y)
        
        # Retrain base models on all data
        for regr in self.regr_:
            regr.fit(X, y)

        return self

    def predict(self, X):
        meta_features = np.column_stack([
            regr.predict(X) for regr in self.regr_
        ])

        if self.use_features_in_secondary:
            return self.meta_regr_.predict(np.hstack((X, meta_features)))
        else:
            return self.meta_regr_.predict(meta_features)

In [10]:
class AveragingRegressor(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, regressors):
        self.regressors = regressors
        
    def fit(self, X, y):
        self.regr_ = [clone(x) for x in self.regressors]
        
        for regr in self.regr_:
            regr.fit(X , y)
        
        return self
    
    def predict(self, X):
        predictions = np.column_stack([regr.predict(X) for regr in self.regr_])
        return np.mean(predictions, axis=1)

In [12]:
en = make_pipeline(RobustScaler(), SelectFromModel(Lasso(alpha=0.03)), ElasticNet(alpha=0.001, l1_ratio = 0.1))
rf = RandomForestRegressor(n_estimators=250,min_samples_split=11, max_depth=5)
et = ExtraTreesRegressor(n_estimators=100,min_samples_split=11, max_depth=5)
xgbm = xgb.sklearn.XGBRegressor(max_depth=5, learning_rate=0.025, subsample=0.9, colsample_bytree=0.4,\
                                base_score=y_mean,objective='reg:linear',n_estimators=1000)

In [24]:
stack_avg = StackingCVRegressorAveraged((en,rf,et), ElasticNet(l1_ratio=0.1, alpha=1.4))
stack_with_feats = StackingCVRegressorRetrained((en,rf,et), xgbm, use_features_in_secondary=True)
stack_retrain = StackingCVRegressorRetrained((en, rf, et), ElasticNet(l1_ratio=0.1, alpha=1.4))
averaged = AveragingRegressor((en,rf,et,xgbm))

In [13]:
results = cross_val_score(en, train.values, y_train, cv=5, scoring='r2')
print("ElasticNet score: %.4f (%.4f)" % (results.mean(), results.std()))

/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


ElasticNet score: 1.0000 (0.0000)


/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [16]:
results = cross_val_score(rf, train.values, y_train, cv=5, scoring='r2')
print("RandomForest score: %.4f (%.4f)" % (results.mean(), results.std()))

RandomForest score: 0.9986 (0.0005)


In [18]:
results = cross_val_score(et, train.values, y_train, cv=5, scoring='r2')
print("ExtraTrees score: %.4f (%.4f)" % (results.mean(), results.std()))

ExtraTrees score: 0.9807 (0.0025)


In [22]:
results = cross_val_score(xgbm, train.values, y_train, cv=5, scoring='r2')
print("XGBoost score: %.4f (%.4f)" % (results.mean(), results.std()))

XGBoost score: 0.9883 (0.0028)


In [25]:
results = cross_val_score(averaged, train.values, y_train, cv=5, scoring='r2')
print("Averaged base models score: %.4f (%.4f)" % (results.mean(), results.std()))

/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


Averaged base models score: 0.9967 (0.0006)


In [47]:
results = cross_val_score(stack_with_feats, train.values, y_train, cv=5, scoring='r2')
print("Stacking (with primary feats) score: %.4f (%.4f)" % (results.mean(), results.std()))

NameError: name 'train_index' is not defined

In [ ]:
results = cross_val_score(stack_retrain, train.values, y_train, cv=5, scoring='r2')
print("Stacking (retrained) score: %.4f (%.4f)" % (results.mean(), results.std()))

In [33]:
results = cross_val_score(stack_avg, train.values, y_train, cv=5, scoring='r2')
print("Stacking (averaged) score: %.4f (%.4f)" % (results.mean(), results.std()))

NameError: name 'kfold' is not defined

In [34]:
gh = KFold(n_splits=5, random_state=42)